<a href="https://colab.research.google.com/github/TobyW28/FTL-TRANS/blob/main/split_into_chunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/My Drive/train.csv")
val_df = pd.read_csv("/content/drive/My Drive/val.csv")
test_df = pd.read_csv("/content/drive/My Drive/test.csv")


In [ ]:

def split_into_chunks(df, max_len):
    input_ids = df.Input_ID.apply(lambda x: x[1:-1].replace(' ', '').split(','))
    df_len = len(df)
    Adm_ID, Note_ID, Input_ID, Label, chartdate, charttime = [], [], [], [], [], []
    for i in tqdm(range(df_len)):
        x = input_ids[i]
        n = int(len(x) / (max_len - 2))
        for j in range(n):
            Adm_ID.append(df.Adm_ID[i])
            Note_ID.append(df.Note_ID[i])
            sub_ids = x[j * (max_len - 2): (j + 1) * (max_len - 2)]
            sub_ids.insert(0, '101')
            sub_ids.append('102')
            Input_ID.append(' '.join(sub_ids))
            Label.append(df.Label[i])
            chartdate.append(df.chartdate[i])
            charttime.append(df.charttime[i])
        if len(x) % (max_len - 2) > 10:
            Adm_ID.append(df.Adm_ID[i])
            Note_ID.append(df.Note_ID[i])
            sub_ids = x[-((len(x)) % (max_len - 2)):]
            sub_ids.insert(0, '101')
            sub_ids.append('102')
            Input_ID.append(' '.join(sub_ids))
            Label.append(df.Label[i])
            chartdate.append(df.chartdate[i])
            charttime.append(df.charttime[i])
    new_df = pd.DataFrame({'Adm_ID': Adm_ID,
                           'Note_ID': Note_ID,
                           'Input_ID': Input_ID,
                           'Label': Label,
                           'chartdate': chartdate,
                           'charttime': charttime})
    new_df = new_df.astype({'Adm_ID': 'int64', 'Note_ID': 'int64', 'Label': 'int64'})
    return new_df

In [ ]:
from tqdm import trange, tqdm

In [ ]:
new_train_df = split_into_chunks(train_df, 128)
new_val_df = split_into_chunks(val_df, 128)
new_test_df = split_into_chunks(test_df, 128)

train_result = new_train_df.Label.value_counts()
val_result = new_val_df.Label.value_counts()
test_result = new_test_df.Label.value_counts()

100%|██████████| 9614/9614 [00:01<00:00, 7886.79it/s]


In [ ]:
new_train_df.to_csv('/content/drive/My Drive/train_chunk.csv', index=False)
new_val_df.to_csv('/content/drive/My Drive/val_chunk.csv', index=False)
new_test_df.to_csv('/content/drive/My Drive/test_chunk.csv', index=False)

In [ ]:
test_df

,Adm_ID,Note_ID,TEXT,Input_ID,Label,chartdate,charttime
0,148959.0,933.0,date of birth: ...,"[3058, 1997, 4182, 1024, 3348, 1024, 1049, 232...",0.0,2122-01-24,NaN
1,148959.0,109600.0,sinus tachycardia. left axis deviation. clockw...,"[8254, 2271, 11937, 11714, 11522, 2401, 1012, ...",0.0,2122-01-15,NaN
2,148959.0,109601.0,sinus rhythm left axis deviation since previou...,"[8254, 2271, 6348, 2187, 8123, 24353, 2144, 30...",0.0,2122-01-14,NaN
3,148959.0,109602.0,sinus tachycardia left axis deviation since pr...,"[8254, 2271, 11937, 11714, 11522, 2401, 2187, ...",0.0,2122-01-13,NaN
4,148959.0,109603.0,sinus rhythm. left anterior fascicular block....,"[8254, 2271, 6348, 1012, 2187, 15099, 6904, 11...",0.0,2122-01-06,NaN
...,...,...,...,...,...,...,...
9609,174749.0,1291497.0,"micu npn 7a-7p neuro: pt. aao x 3, talking to ...","[23025, 2226, 27937, 2078, 1021, 2050, 1011, 1...",0.0,2127-04-24,2127-04-24 19:57:00
9610,174749.0,1291498.0,"nursing progress note neuro: awake, alert, ple...","[8329, 5082, 3602, 11265, 10976, 1024, 8300, 1...",0.0,2127-04-25,2127-04-25 06:01:00
9611,174749.0,1291495.0,nursing admission note pt is a 49 yr old man w...,"[8329, 9634, 3602, 13866, 2003, 1037, 4749, 10...",0.0,2127-04-24,2127-04-24 05:29:00
9612,174749.0,1291496.0,resp. care note pt returned from tips procedur...,"[24501, 2361, 1012, 2729, 3602, 13866, 2513, 2...",0.0,2127-04-24,2127-04-24 06:31:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00


In [ ]:
pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=a3b100830070f615d183e41506c124e9240162d2f122ce1907336fdf74bfcb47
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
